In [1]:
from ipynb.fs.full.TetrisGame import game

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import copy

In [3]:
### Definimos la arquitectura que tendrá cada individuo de la poblacion de nuestro algoritmo genético.
class Gen_net(nn.Module):
    '''
    Definimos un modelo de red neuronal sobre el cual podamos 
    evaluar el algoritmo genético.
    '''
    def __init__(self, in_features, hidden_1, hidden_2, out_features):
        super(Gen_net,self).__init__()
        '''
        Define la estructura de la red como se define en el articulo.
        Ten en cuenta que, dado que no usamos el mismo conjunto de datos
        los valores de in_features y out_features se verán afectados.
        Agregar como parámetros estos valores te puede ayudar a reconstruir
        la red a partir de un cromosoma.
        '''
        ## Modela la conexion entre la capa de entrada y la primera capa oculta
        self.fc1 = nn.Linear(in_features = in_features, out_features = hidden_1, bias=True)
        
        ## Modela la conexion entre la primera capa oculta y la segunda capa oculta
        self.fc2 = nn.Linear(in_features = hidden_1, out_features = hidden_2, bias=True)
        
        # Modela la conexion entre la segunda capa oculta y la capa de salida.
        self.fc3 = nn.Linear(in_features = hidden_2, out_features = out_features, bias=True)
        
        ## Construccion del cromosoma de esta red.
        self.chromosomeSize = in_features*(hidden_1 + hidden_2 + out_features)
        self.chromosome = np.array([0.0]*self.chromosomeSize)
        self.chromosome = np.random.normal(size = self.chromosomeSize)
        self.build_from_chromosome()
        
    def build_from_chromosome(self):
        '''
        Define una función para que a partir de un cromosoma modifiques los 
        pesos de la red.        
        '''
        ### La forma de encoding que estoy usando es la forma que se usó para pasar vectores a pesos y de recuperar
        ### las matrices de pesos a partir de un vector que se usó en la práctica de MNIST.
        w0 = self.fc1.weight
        w1 = self.fc2.weight
        w2 = self.fc3.weight
        aux = 0
        for i in range(len(self.fc3.weight)):
            for j in range(len(self.fc3.weight[0])):
                w2[i][j] = self.chromosome[aux]
                aux += 1
        for i in range(len(self.fc2.weight)):
            for j in range(len(self.fc2.weight[0])):
                w1[i][j] = self.chromosome[aux]
                aux += 1
        for i in range(len(self.fc1.weight)):
            for j in range(len(self.fc1.weight[0])):
                w0[i][j] = self.chromosome[aux]
                aux += 1
                
        self.fc1.weight = torch.nn.Parameter(w0)
        self.fc2.weight = torch.nn.Parameter(w1)
        self.fc3.weight = torch.nn.Parameter(w2)
       
    def forward(self, X):
        '''
        Define la función de feed forward, a partir de los datos
        de X. Usa una sigmoide entre las capas.
        '''
        X = torch.Tensor(X)
        ## Tomamos los datos de entrada de la capa de entrada y aplicamos la transformación lineal correspondiente
        primer_capa_oculta = torch.sigmoid(self.fc1(X))
        
        # Ahora en primer_capa_oculta tenemos los valores resultantes de pasar los datos de entrada por la primera capa
        # oculta, ahora necesitamos procesar en la siguiente capa oculta
        segunda_capa_oculta = torch.sigmoid(self.fc2(primer_capa_oculta))
        
        # Obtenemos los valores de la capa de salida
        salida = torch.sigmoid(self.fc3(segunda_capa_oculta))
        return salida
        
        
    ## Mutación de pesos sin sesgo
    def unbiased_mutate_weights(self):
        ## Para cada entrada en el cromosoma, remuestrear el valor con probabilidad p = 0.1
        prob = 0.1
        for i in range(self.chromosomeSize):
            random = np.random.rand()
            if random < prob:
                self.chromosome[i] = np.random.normal(size = 1)
                
        ## Mutación de pesos con sesgo
    def biased_mutate_weights(self):
        ## Para cada entrada en el cromosoma, agregar ruido al valor con probabilidad p = 0:1.
        prob = 0.1
        for i in range(self.chromosomeSize):
            random = np.random.rand()
            if random < prob:
                self.chromosome[i] += (1/1)*np.random.normal(size = 1)

In [8]:
class Player():
    def __init__(self):
        self.playerNN = Gen_net(200, 30, 6, 6)
        self.fitness = 0
        self.score = 0
        tetris = game()
        self.currentGame = tetris
        self.isGameOver = False
        
    def calculateFitness(self):
        self.fitness = self.currentGame.board.points
        
    def makeMove(self):
        self.currentGame.receiveFromNN(self.playerNN)
        self.isGameOver = self.currentGame.board.isGameOver
        

In [9]:
class Population():
    def __init__(self, size):
        self.players = []
        self.fitnessSum = 0
        self.bestPlayer = None
        self.generation = 0
        self.moves = 0
        for i in range(size):
            player = Player()
            self.players.append(player)
            
    def setBestPlayer(self):
        self.bestPlayer = self.players[0]
        for i in range(len(self.players)):
            if self.bestPlayer.fitness < self.players[i].fitness:
                self.bestPlayer = copy.copy(self.players[i])
                
    def selectPlayer(self):
        randomSelection = np.random.randint(self.fitnessSum)
        auxSum = 0
        for i in range(len(self.players)):
            auxSum += self.players[i].fitness
            if auxSum > randomSelection:
                return self.players[i]
            
    def calculateFitnessPlayer(self):
        for i in range(len(self.players)):
            self.players[i].calculateFitness()
            
    def calculateFitnessSum(self):
        for i in range(len(self.players)):
            self.fitnessSum += self.players[i].fitness
            
    def allPlayersGameOver(self):
        aux = 0
        for i in range(len(self.players)):
            if self.players[i].isGameOver == True:
                aux += 1
        if aux == len(self.players):
            return True
        else:
            return False
    
    def naturalSelection(self):
        self.calculateFitnessPlayer()
        self.calculateFitnessSum()
        print(self.fitnessSum)
        self.setBestPlayer()
        playersInNextGen = []
        bestPlayerNextGen = copy.copy(self.bestPlayer)
        new_game = game()
        bestPlayerNextGen.currentGame = new_game
        bestPlayerNextGen.isGameOver = False
        playersInNextGen.append(bestPlayerNextGen)
        for i in range(0, len(self.players) - 1):
                parent = self.selectPlayer()
                child = copy.copy(parent)
                child.isGameOver = False
                newGame = game()
                child.currentGame = newGame
                child.playerNN.biased_mutate_weights()
                child.playerNN.build_from_chromosome()
                playersInNextGen.append(child)
        self.players = playersInNextGen
        self.calculateFitnessPlayer()
        self.generation += 1
        self.moves = 0
        self.fitnessSum = 0

        
    def allMakeMove(self):
        self.moves += 1
        if self.moves%4 == 0:
            for i in range(0, len(self.players)):
                if self.players[i].isGameOver == False:
                    self.players[i].currentGame.board.movePieceDown()
        for i in range(0, len(self.players)):
            if self.players[i].isGameOver == False:
                self.players[i].makeMove()

                    
                
                
    def train(self, n_generations, n_moves):
        for i in range(n_generations):
            for j in range(n_moves):
                self.allMakeMove()
            self.naturalSelection()
            
    def trainToGameOver(self, n_generations):
        for i in range(n_generations):
            while self.allPlayersGameOver() == False:
                self.allMakeMove()
            self.naturalSelection()
        

In [12]:
### Creamos una población.
poblacion = Population(30)
for i in range(40):
    poblacion.allMakeMove()

In [13]:
poblacion.trainToGameOver(50)

511
1146
1934
1743
1939
2006
1669
1949
1884
991
1199
1722
199
309
364
282
102
282
8
63
1359
2880
1833
1  lines were completed
1088
2811
2873
2749
1  lines were completed
1  lines were completed
3833
614
1173
1879
2248
1  lines were completed
3192
2462
1843
2265
1932
1  lines were completed
1960
944
828
1409
1378
1093
747
1292
606
683
346
395
306


In [18]:
for i in range(100):
    poblacion.allMakeMove()
    
poblacion.players[6].currentGame.board.showBoard2()

[[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'o︎' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' 'o︎' 'o︎' 'o︎' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' 'o︎' 'o︎' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' 'o︎' 'o︎' 'o︎' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' 'o︎' 'o︎' 'o︎' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' 'o︎' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' 'o︎' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' 'o︎' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' 'o︎' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' 'o︎' 'o︎' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' 'o︎' 'o︎' ' ' ' ' ' ']
 [' ' ' ' ' ' 'o︎' 'o︎' 'o︎' 'o︎' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' 'o︎' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' 'o︎' 'o︎' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' 'o︎' ' ' ' ' ' ' ' ' ' ']]
Next Piece:
[[' ' ' ' ' ' ' ']
 [' ' 'o' 'o' ' ']
 [' ' 'o' 'o' ' ']
 [' ' ' ' ' ' ' ']]
Points:  2


In [19]:
poblacion.calculateFitnessPlayer()
poblacion.setBestPlayer()
poblacion.bestPlayer.currentGame.board.showBoard2()


[[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' 'o︎' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' 'o︎' 'o︎' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' 'o︎' 'o︎' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' 'o︎' 'o︎' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' 'o︎' 'o︎' 'o︎' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' 'o︎' 'o︎' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'o︎' 'o︎' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'o︎' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'o︎' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'o︎' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' 'o︎' 'o︎' 'o︎' 'o︎' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' 'o︎' 'o︎' 'o︎' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' 'o︎' 'o︎' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' 'o︎' 'o︎' 'o︎' 'o︎' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' 'o︎' 'o︎' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'o︎' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'o︎' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'o︎' 'o︎' ' ' ' ']]
Next Piece:
[[' ' 'o' ' ' ' ']
 [' ' 'o' 'o' 'o']
 [' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ']]
Points:  31
